# MS 2.2- Group 10

**Members**: Niklas Grüner (12217059), Konstantin Unterweger (12222169), Martin Harhammer (12221683)


The following sections describe and implement an attempt to Audio Identification. 

## Imports

In [1]:
# imports
import os, sys
import numpy as np
from numba import jit
import librosa
#from scipy import signal
from scipy import ndimage
from matplotlib import pyplot as plt
import IPython.display as ipd
import time

sys.path.append('..')
import libfmp.b
import libfmp.c2
import libfmp.c6
from IPython.display import clear_output
from collections import defaultdict
import math
import matplotlib.pyplot as plt

%matplotlib inline

## Utility functions

Here we define all functions that, we later need to compute spectrograms, constellation maps, the hashes and perform the matching

In [2]:
def load_filenames(directory):
    filenames = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        # Create the full path
        file_path = os.path.join(directory, filename)
        
        # Check if it is a file (not a directory)
        if os.path.isfile(file_path):
            # Add to the dictionary, using the filename as the key
            filenames.append(file_path)

    return filenames

In [3]:
def compute_spectrogram(fn_wav, Fs=22050, N=2048, H=1024, bin_max=128, frame_max=None, duration=None):
    x, Fs = librosa.load(fn_wav, sr=Fs, duration=30)
    x_duration = len(x) / Fs
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann')
    if bin_max is None:
        bin_max = X.shape[0]
    if frame_max is None:
        frame_max = X.shape[0]
    Y = np.abs(X[:bin_max, :frame_max])
    return Y

In [4]:
def compute_constellation_map(Y, dist_freq=7, dist_time=7, thresh=0.01):
    """Compute constellation map (implementation using image processing)

    Notebook: C7/C7S1_AudioIdentification.ipynb

    Args:
        Y (np.ndarray): Spectrogram (magnitude)
        dist_freq (int): Neighborhood parameter for frequency direction (kappa) (Default value = 7)
        dist_time (int): Neighborhood parameter for time direction (tau) (Default value = 7)
        thresh (float): Threshold parameter for minimal peak magnitude (Default value = 0.01)

    Returns:
        Cmap (np.ndarray): Boolean mask for peak structure (same size as Y)
    """
    result = ndimage.maximum_filter(Y, size=[2*dist_freq+1, 2*dist_time+1], mode='constant')
    Cmap = np.logical_and(Y == result, result > thresh)
    return Cmap

In [5]:
from concurrent.futures import ThreadPoolExecutor

def compute_constellation_map_single(args):
    """Compute the constellation map for a single file."""
    filename, dist_freq, dist_time = args
    spectrogram = compute_spectrogram(filename)  # Perform I/O and computation
    constellation_map = compute_constellation_map(spectrogram, dist_freq, dist_time)
    return filename, constellation_map

def compute_constellation_maps(filenames, dist_freq, dist_time):
    """Compute constellation maps using multithreading."""
    # Prepare arguments for each file
    args = [(filename, dist_freq, dist_time) for filename in filenames]
    
    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor() as executor:
        results = executor.map(compute_constellation_map_single, args)
    
    # Convert results to a dictionary
    Cmaps = dict(results)
    return Cmaps

In [6]:
def build_database(
    directory,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    Build a fingerprint database from a dictionary of spectrograms.
    
    Args:
        cmaps_D (dict): {track_name: 2D numpy array (boolean spectrogram)}
        time_min_offset, time_max_offset, freq_min_offset, freq_max_offset (int): 
            Bounds for the "target zone" around an anchor point.
    
    Returns:
        database (defaultdict): {hash: [(track_name, time_offset), ...]}
    """
    tracks = load_filenames(directory) # load all track filenames
    cmaps_D = compute_constellation_maps(tracks, 11, 3) # store the computed constellation maps for each configuration.
    
    database = defaultdict(list)
    
    for track_name, cmap in cmaps_D.items():
        # Convert the boolean map to a list of points (freq, time), sorted by time
        point_list = sorted(np.argwhere(cmap).tolist(), key=lambda x: x[1])

        for anchor in point_list:
            # Find all points in the anchor's target zone
            target_points = get_target_zone_points(
                anchor, 
                point_list,
                time_min_offset=time_min_offset,
                time_max_offset=time_max_offset,
                freq_min_offset=freq_min_offset,
                freq_max_offset=freq_max_offset
            )

            # Compute hash for each (anchor, target) pair
            for target_point in target_points:
                h = compute_hash(anchor, target_point)
                database[h].append((track_name, anchor[1]))
    
    return database


In [7]:
def compute_hash(anchor, target):
    """Generate a 32-bit hash."""
    f1, t1 = anchor
    f2, t2 = target
    dt = t2 - t1
    return (f1 & 0x3FF) | ((f2 & 0x3FF) << 10) | ((dt & 0xFFF) << 20)



In [8]:
def get_target_zone_points(anchor, point_list,
                           time_min_offset, time_max_offset,
                           freq_min_offset, freq_max_offset):
    """
    Finds points in 'point_list' that lie within time [t1 + time_min_offset, t1 + time_max_offset]
    and frequency [f1 + freq_min_offset, f1 + freq_max_offset].
    
    anchor: (f1, t1) or (t1, f1) – whichever convention you are using.
    point_list: list of (f, t) or (t, f) – must be sorted by the time coordinate if we want to break early.
    time_min_offset, time_max_offset: how far in time we look relative to anchor's time t1.
    freq_min_offset, freq_max_offset: how far in frequency we look relative to anchor's freq f1.
    """
    f1, t1 = anchor
    
    # Time bounds
    t_min = t1 + time_min_offset
    t_max = t1 + time_max_offset
    
    # Frequency bounds
    f_min = f1 + freq_min_offset
    f_max = f1 + freq_max_offset
    
    target_zone_points = []
    for (f2, t2) in point_list:
        # If the list is sorted by time and t2 > t_max, we can break early.
        if t2 > t_max:
            break
        
        if t_min < t2 <= t_max and f_min <= f2 <= f_max:
            target_zone_points.append((f2, t2))
    
    return target_zone_points


In [9]:
import re
import random

def extract_numeric_id(filename):
    """
    Example: 
      - 'queries/1269810_original.mp3' => '1269810'
      - 'tracks/1269810.mp3'           => '1269810'
    Adjust to your naming conventions as needed.
    """
    match = re.search(r'(\d+)', filename)
    return match.group(1) if match else None

In [10]:
def find_best_match_for_query(
    query_name,
    cmap,
    database,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    Given a query's spectrogram cmap (2D Boolean array),
    find the best matching track in the database of fingerprints.

    Returns:
        best_track: str or None
        best_delta: int or None
        best_count: int
        point_list: list of non-zero (freq,time) points in the query
    """
    # Convert the boolean array to a list of points (freq, time)
    point_list = sorted(np.argwhere(cmap).tolist(), key=lambda x: x[1])

    # Dictionary of track_name -> (offset -> count)
    matches = defaultdict(lambda: defaultdict(int))
    
    # For each point in the query, find matching points in the database
    for anchor in point_list:
        target_points = get_target_zone_points(
            anchor,
            point_list,
            time_min_offset=time_min_offset,
            time_max_offset=time_max_offset,
            freq_min_offset=freq_min_offset,
            freq_max_offset=freq_max_offset
        )

        for target_point in target_points:
            h = compute_hash(anchor, target_point)

            # If our hash is found in the database
            if h in database:
                # database[h] = list of (track_name, track_offset)
                for track_name, track_offset in database[h]:
                    delta_offset = track_offset - anchor[1]
                    matches[track_name][delta_offset] += 1
    
    # Find the best match with the highest count
    best_track = None
    best_delta = None
    best_count = 0
    
    for track_name, offset_counts in matches.items():
        for delta_offset, count in offset_counts.items():
            if count > best_count:
                best_count = count
                best_track = track_name
                best_delta = delta_offset

    return best_track, best_delta, best_count, point_list

In [11]:

def match_queries(
    directory,
    database,
    time_min_offset=5,
    time_max_offset=30,
    freq_min_offset=-15,
    freq_max_offset=15
):
    """
    - Matches each query in cmaps_Q to the best track in the database.
    :param cmaps_Q: dict {query_name: 2D numpy array (spectrogram boolean map)}
    :param database: dict {hash: [(track_name, track_offset), ...]}
    """

    queries = load_filenames(directory) # load all track filenames
        
    results = []
    num_correct = 0
    
    # Collect queries in a list to iterate consistently
    total_queries = len(queries)

    

    for query_name in queries:
        starttime = time.time()
        
        _, cmap = compute_constellation_map_single((query_name, 11, 3))       

        
        best_track, best_delta, best_count, point_list = find_best_match_for_query(
            query_name,
            cmap,
            database,
            time_min_offset=time_min_offset,
            time_max_offset=time_max_offset,
            freq_min_offset=freq_min_offset,
            freq_max_offset=freq_max_offset
        )

        endtime = time.time()
        

        query_id = extract_numeric_id(query_name)
        track_id = extract_numeric_id(best_track) if best_track else None
        correct = (query_id == track_id)
        if correct:
            num_correct += 1

        # Store the result
        results.append((query_name, best_track, best_delta, best_count, correct, (endtime-starttime)*1000))

    # ----------------------------------------------------------------------
    # Print final results
    # ----------------------------------------------------------------------
    accuracy = num_correct / total_queries if total_queries else 0
    print("\nMATCHING RESULTS:")
    sum_duration = 0
    for qname, tname, offset, count, is_correct, duration in results:
        print(f"Query: {qname} => Best match: {tname}, Time: {duration} ms, Offset: {offset}, Count: {count}, Correct: {is_correct}")
        sum_duration += duration

    print(f"\nCorrect matches: {num_correct}/{total_queries}")
    print(f"Average query time: {sum_duration/len(results)} ms")
    print(f"Accuracy: {accuracy*100:.2f}%")

    return results

# Task 1

## Define configurations

In [12]:
config_1 = {
    "time_min_offset": 5,
    "time_max_offset": 30,
    "freq_min_offset": -15,
    "freq_max_offset": 15
}

config_2 = {
    "time_min_offset": 0,
    "time_max_offset": 30,
    "freq_min_offset": 0,
    "freq_max_offset": 25
}

config_3 = {
    "time_min_offset": 10,
    "time_max_offset": 30,
    "freq_min_offset": 0,
    "freq_max_offset": 30
}

config_4 = {
    "time_min_offset": 5,
    "time_max_offset": 10,
    "freq_min_offset": -5,
    "freq_max_offset": 5
}

## Build the database

In [13]:
s = time.time()
database_1 = build_database("tracks", **config_1)
e = time.time()
print(e-s)

115.06431126594543


In [14]:
database_2 = build_database("tracks", **config_2)

In [15]:
database_3 = build_database("tracks", **config_3)

In [16]:
database_4 = build_database("tracks", **config_4)

# Task 2: Audio Identification

### Match all test queries

In [17]:
s = time.time()
matches_1 = match_queries("queries", database_1, **config_1)
e = time.time()
print(e-s)


MATCHING RESULTS:
Query: queries/1269810_original.mp3 => Best match: tracks/1269810.low.mp3, Time: 548.0213165283203 ms, Offset: 128, Count: 511, Correct: True
Query: queries/1400510_noise.mp3 => Best match: tracks/1392703.low.mp3, Time: 93.22142601013184 ms, Offset: 176, Count: 5, Correct: False
Query: queries/887210_mobile.mp3 => Best match: tracks/887210.low.mp3, Time: 244.58956718444824 ms, Offset: 90, Count: 51, Correct: True
Query: queries/1269810_noise.mp3 => Best match: tracks/1269810.low.mp3, Time: 555.9771060943604 ms, Offset: 128, Count: 355, Correct: True
Query: queries/1400510_mobile.mp3 => Best match: tracks/1121504.low.mp3, Time: 71.79617881774902 ms, Offset: 192, Count: 10, Correct: False
Query: queries/1084710_mobile.mp3 => Best match: tracks/822509.low.mp3, Time: 65.64617156982422 ms, Offset: 65, Count: 9, Correct: False
Query: queries/119410_original.mp3 => Best match: tracks/119410.low.mp3, Time: 168.05315017700195 ms, Offset: 194, Count: 279, Correct: True
Query: 

In [18]:
matches_2 = match_queries("queries", database_2, **config_2)


MATCHING RESULTS:
Query: queries/1269810_original.mp3 => Best match: tracks/1269810.low.mp3, Time: 537.8031730651855 ms, Offset: 128, Count: 450, Correct: True
Query: queries/1400510_noise.mp3 => Best match: tracks/1418110.low.mp3, Time: 89.11252021789551 ms, Offset: 382, Count: 8, Correct: False
Query: queries/887210_mobile.mp3 => Best match: tracks/887210.low.mp3, Time: 2213.637351989746 ms, Offset: 90, Count: 46, Correct: True
Query: queries/1269810_noise.mp3 => Best match: tracks/1269810.low.mp3, Time: 543.3399677276611 ms, Offset: 128, Count: 312, Correct: True
Query: queries/1400510_mobile.mp3 => Best match: tracks/479909.low.mp3, Time: 71.73800468444824 ms, Offset: 257, Count: 8, Correct: False
Query: queries/1084710_mobile.mp3 => Best match: tracks/822509.low.mp3, Time: 56.75172805786133 ms, Offset: 65, Count: 9, Correct: False
Query: queries/119410_original.mp3 => Best match: tracks/119410.low.mp3, Time: 186.39898300170898 ms, Offset: 194, Count: 286, Correct: True
Query: que

In [19]:
matches_3 = match_queries("queries", database_3, **config_3)


MATCHING RESULTS:
Query: queries/1269810_original.mp3 => Best match: tracks/1269810.low.mp3, Time: 436.1729621887207 ms, Offset: 128, Count: 397, Correct: True
Query: queries/1400510_noise.mp3 => Best match: tracks/1304400.low.mp3, Time: 69.36812400817871 ms, Offset: 271, Count: 4, Correct: False
Query: queries/887210_mobile.mp3 => Best match: tracks/887210.low.mp3, Time: 187.75177001953125 ms, Offset: 90, Count: 34, Correct: True
Query: queries/1269810_noise.mp3 => Best match: tracks/1269810.low.mp3, Time: 433.0439567565918 ms, Offset: 128, Count: 277, Correct: True
Query: queries/1400510_mobile.mp3 => Best match: tracks/1110704.low.mp3, Time: 57.730913162231445 ms, Offset: 187, Count: 6, Correct: False
Query: queries/1084710_mobile.mp3 => Best match: tracks/822509.low.mp3, Time: 44.99506950378418 ms, Offset: 65, Count: 5, Correct: False
Query: queries/119410_original.mp3 => Best match: tracks/119410.low.mp3, Time: 143.30697059631348 ms, Offset: 194, Count: 221, Correct: True
Query: 

In [20]:
matches_4 = match_queries("queries", database_4, **config_4)


MATCHING RESULTS:
Query: queries/1269810_original.mp3 => Best match: tracks/1269810.low.mp3, Time: 126.08599662780762 ms, Offset: 128, Count: 53, Correct: True
Query: queries/1400510_noise.mp3 => Best match: tracks/388810.low.mp3, Time: 39.014577865600586 ms, Offset: 375, Count: 2, Correct: False
Query: queries/887210_mobile.mp3 => Best match: tracks/1414102.low.mp3, Time: 54.9471378326416 ms, Offset: 13, Count: 4, Correct: False
Query: queries/1269810_noise.mp3 => Best match: tracks/1269810.low.mp3, Time: 121.99926376342773 ms, Offset: 128, Count: 34, Correct: True
Query: queries/1400510_mobile.mp3 => Best match: tracks/159207.low.mp3, Time: 24.714946746826172 ms, Offset: 272, Count: 2, Correct: False
Query: queries/1084710_mobile.mp3 => Best match: tracks/342300.low.mp3, Time: 23.99444580078125 ms, Offset: 340, Count: 2, Correct: False
Query: queries/119410_original.mp3 => Best match: tracks/119410.low.mp3, Time: 48.4311580657959 ms, Offset: 194, Count: 32, Correct: True
Query: quer

# Task 3: Scale up

# Task 4: Report